In [4]:
import sqlite3
import pandas as pd
from datetime import datetime
import os

In [2]:
def get_tables(db_path="stock.db"):

    conn = None
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        results = cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        for row in results:
            print(row)
        conn.commit()
        print("Query executed successfully.")
    except sqlite3.Error as e:
        print(f"Database error: {e}")
    finally:
        if conn:
            conn.close()

In [12]:
get_tables()

('stock_index_price_daily',)
('conversation_history',)
Query executed successfully.


In [5]:
def execute_query( query, db_path="stock.db"):

    conn = None
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        results = cursor.execute(query)
        for row in results:
            print(row)
        conn.commit()
        print("Query executed successfully.")
    except sqlite3.Error as e:
        print(f"Database error: {e}")
    finally:
        if conn:
            conn.close()

execute_query("SELECT name FROM sqlite_master WHERE type='table';")

('conversation_history',)
('stock_index_price_daily',)
Query executed successfully.


In [16]:
execute_query("SELECT * FROM stock_index_price_daily limit 1;")

('NIFTY AUTO', 12700, 12700.05, 12593.2, 12611.7, '2022-12-30')
Query executed successfully.


In [ ]:
def insert_stock_data_from_file(db_path, csv_path, cursor):
    """
    Reads stock data from a single CSV file and inserts it into the 'stock_index_price_daily' table.
    """
    try:
        # Read CSV data
        df = pd.read_csv(csv_path)

        # Prepare data for insertion
        data_to_insert = []
        for _, row in df.iterrows():
            # Convert date format
            try:
                date_obj = datetime.strptime(row['Date'], '%d %b %Y')
                date_key = date_obj.strftime('%Y-%m-%d')
            except ValueError:
                print(f"Could not parse date: {row['Date']} in {os.path.basename(csv_path)}. Skipping row.")
                continue

            data_to_insert.append((
                row['Index Name'],
                row['Open'],
                row['High'],
                row['Low'],
                row['Close'],
                date_key
            ))

        # Insert data into the table
        cursor.executemany("""
            INSERT INTO stock_index_price_daily (index_name, open, high, low, close, date_key)
            VALUES (?, ?, ?, ?, ?, ?)
        """, data_to_insert)
        print(f"Successfully inserted {len(data_to_insert)} rows from {os.path.basename(csv_path)}.")
    except FileNotFoundError:
        print(f"Error: The file '{csv_path}' was not found.")
    except Exception as e:
        print(f"An error occurred while processing {csv_path}: {e}")

In [ ]:
def insert_stock_data_from_directory(db_path, directory_path):
    """
    Reads stock data from all CSV files in a directory and inserts it into the 'stock_index_price_daily' table.
    """
    conn = None
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # Drop the table if it exists to ensure a clean slate
        cursor.execute("DROP TABLE IF EXISTS stock_index_price_daily")

        # Create table
        cursor.execute("""
            CREATE TABLE stock_index_price_daily (
                index_name TEXT,
                open REAL,
                high REAL,
                low REAL,
                close REAL,
                date_key TEXT
            )
        """)

        for filename in os.listdir(directory_path):
            if filename.endswith(".csv"):
                csv_path = os.path.join(directory_path, filename)
                insert_stock_data_from_file(db_path, csv_path, cursor)

        conn.commit()

    except sqlite3.Error as e:
        print(f"Database error: {e}")
    finally:
        if conn:
            conn.close()

if __name__ == "__main__":
    db_path = "stock.db"
    directory_path = "Market Data/NiftyChemicals"
    insert_stock_data_from_directory(db_path, directory_path)

In [9]:
execute_query("SELECT distinct index_name from stock_index_price_daily;")

('NIFTY FINANCIAL SERVICES',)
('NIFTY AUTO',)
('NIFTY CHEMICALS',)
('NIFTY CONSUMER DURABLES',)
('NIFTY BANK',)
Query executed successfully.
